In [1]:
# =========================================================
# 📘 GeoJSON 행정구역별 landuse 확률 합계 및 평균 집계기
#  - jsonl: 각 타일별 image_id(y_x.png), probs, ADM0/ADM1/ADM2
#  - geojson: CRS84 (lon/lat)
#  - image_id에서 타일 중심 좌표를 계산해 폴리곤 포함 여부 판단
# =========================================================

import json
import math
from collections import defaultdict, Counter
import numpy as np
from shapely.geometry import shape, Point
from shapely.strtree import STRtree

# ---------------------------
# 🔧 유틸 함수
# ---------------------------

def slippy_tile_lonlat_center(x: int, y: int, z: int):
    """Return (lon, lat) of the center of slippy tile (x, y, z) in WGS84."""
    n = 2.0 ** z
    lon_deg = (x + 0.5) / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * (y + 0.5) / n)))
    lat_deg = math.degrees(lat_rad)
    return lon_deg, lat_deg


def parse_xy_from_image_id(image_id: str):
    """
    y_x.png 형식 → y 먼저, x 나중.
    예: '6258_13909.png' → y=6258, x=13909
    """
    stem = image_id.rsplit('.', 1)[0]
    a, b = stem.split('_')
    y = int(a)
    x = int(b)
    return x, y


def clean_name_to_adm12(name: str):
    """
    GeoJSON properties.name 예: '1 Pyonganbuk-do Unsan-gun'
    → 'Pyonganbuk-do', 'Unsan-gun'
    """
    # def safe_split(name, idx=1):
    #     parts = str(name).split()
    #     return parts[idx] if len(parts) > idx else parts[0]
    split_list = name.split()
    if len(split_list) ==1:
        return split_list[0], split_list[0]
    else:
        ADM1, ADM2 = split_list[1], split_list[2] 
        return ADM1, ADM2


def to_dict_from_vec(vec, class_names=None):
    """list(vec) → {label: value}"""
    if class_names:
        return {str(k): float(v) for k, v in zip(class_names, vec)}
    else:
        return {str(i): float(v) for i, v in enumerate(vec)}

# ---------------------------
# 🧩 메인 집계 함수
# ---------------------------

def aggregate(jsonl_path, geojson_path, zoom=14, class_names=None, default_adm0=""):
    # 1️⃣ GeoJSON 읽기
    with open(geojson_path, "r", encoding="utf-8") as f:
        gj = json.load(f)

    geoms = []
    meta = []  # (top_key, adm1, adm2, raw_name)
    for feat in gj.get("features", []):
        geom = shape(feat["geometry"])
        name = feat.get("properties", {}).get("name", "")
        adm1_geo, adm2_geo = clean_name_to_adm12(name)
        top_key = name
        geoms.append(geom)
        meta.append((top_key, adm1_geo, adm2_geo, name))

    # 2️⃣ 공간 인덱스 생성
    tree = STRtree(geoms)
    geom_to_idx = {id(g): i for i, g in enumerate(geoms)}  # 지오메트리→인덱스 매핑

    # 3️⃣ 누적 버킷
    sums = defaultdict(lambda: None)
    counts = Counter()
    adm0_counter = defaultdict(Counter)

    # 4️⃣ JSONL 순회
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            rec = json.loads(line)
            image_id = rec["image_id"]
            probs = rec["probs"]
            adm0_line = rec.get("ADM0", default_adm0)
            # if rec['ADM1'] !='2022':
            #     continue

            x, y = parse_xy_from_image_id(image_id)
            lon, lat = slippy_tile_lonlat_center(x, y, zoom)
            p = Point(lon, lat)

            # ← 여기만 핵심 수정
            cands = tree.query(p)  # 어떤 환경에선 int 인덱스, 어떤 환경에선 geometry를 반환
            if len(cands)==0:
                continue
            # print(cands)
            # print(type(cands[0]))
            # if cands and( isinstance(cands[0], int) or isinstance(cands[0], np.int64)):
                # print('hi2')
            idx_candidates = cands
            # else:
            #     print(cands)
            #     print(type(cands[0]))
            #     idx_candidates = [geom_to_idx[id(g)] for g in cands]

            matched = False
            for idx in idx_candidates:
                candidate = geoms[idx]
                if candidate.contains(p) or candidate.touches(p):
                    top_key, adm1_geo, adm2_geo, rawname = meta[idx]
                    if sums[top_key] is None:
                        sums[top_key] = [0.0] * len(probs)
                    for i, v in enumerate(probs):
                        sums[top_key][i] += float(v)
                    counts[top_key] += 1
                    if adm0_line:
                        adm0_counter[top_key][adm0_line] += 1
                    matched = True
                    break

    # 5️⃣ 결과 조립
    out = {}
    for (top_key, adm1_geo, adm2_geo, rawname) in meta:
        n = counts.get(top_key, 0)
        if n == 0:
            continue
        adm0_iso3 = (adm0_counter[top_key].most_common(1)[0][0]
                     if adm0_counter[top_key] else default_adm0)
        sum_vec = sums[top_key]
        avg_vec = [s / n for s in sum_vec]

        sum_dict = to_dict_from_vec(sum_vec, class_names)
        avg_dict = to_dict_from_vec(avg_vec, class_names)

        out[top_key] = {
            "ADM0": adm0_iso3,
            "ADM1": adm1_geo,
            "ADM2": adm2_geo,
            "landuse": {
                "Landuse_Sum": {
                    "val": sum_dict,
                    "desc": "Sum of landuse ratio",
                    "type": "default",
                    "weight": None,
                },
                "Landuse_Average": {
                    "val": avg_dict,
                    "desc": "Average landuse ratio",
                    "type": "ratio",
                    "weight": float(n),
                },
            },
        }
    return out


In [2]:
# =========================================================
# ⚙️ 실행 예시
# =========================================================

# 파일 경로 지정
jsonl_path = "/home/donggyu/donggyu/RegionalEstimationLLM/*****temporary/PRK_2010_2024_2022.jsonl"
geojson_path = "/home/donggyu/donggyu/RegionalEstimationLLM/*****temporary/2024_AOI_outline_zl14_wo_duplicate.geojson"
output_path = "/home/donggyu/donggyu/RegionalEstimationLLM/*****temporary/PRK_2010_2024_2022.json"

# 줌레벨 (보통 z=14)
zoom = 16

# 클래스 이름 (없으면 자동으로 0,1,2,...)
class_names = [
    "Residential",
    "Agricultural",
    "Commercial",
    "Factory",
    "Mining",
    "Power station",
    "Transportation",
    "Sports",
    "Religious",
    "Water",
    "Others",
]

# ADM0 기본값 (예: NK)
default_adm0 = "PRK"

# 실행
result = aggregate(jsonl_path, geojson_path, zoom=zoom,
                   class_names=class_names, default_adm0=default_adm0)

# 저장
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print(f"[✅] Saved result to {output_path}")
print(f"총 행정구역 수: {len(result)}")


[✅] Saved result to /home/donggyu/donggyu/RegionalEstimationLLM/*****temporary/PRK_2010_2024_2022.json
총 행정구역 수: 23
